In [4]:
### #Loads the best_model after schnet training.
#user evaluates the model on the QM9 database

import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
import pandas as pd
import scipy.linalg as la
from schnetpack.datasets import QM9

# Load best model where it was stored after training

model = torch.load("./qm9tut/best_model", map_location=torch.device('cpu'))

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file 
train, val, test = spk.data.train_test_split(qm9data, split_file='./qm9tut/split.npz')


device = 'cpu'

converter = spk.data.AtomsConverter(device=device)

#Control excel sheets to output data

from xlwt import Workbook

wb1 = Workbook()
wb2 = Workbook()
wb3 = Workbook()
wb4 = Workbook()
wb5 = Workbook()
wb6 = Workbook()
wb7 = Workbook()
wb8 = Workbook()
wb9 = Workbook()
wb10 = Workbook()
wb11 = Workbook()
wb12 = Workbook()
wb13 = Workbook()

sheet1 = wb1.add_sheet('Sheet1')
sheet2 = wb2.add_sheet('Sheet2')
sheet3 = wb3.add_sheet('Sheet3')
sheet4 = wb4.add_sheet('Sheet4')
sheet5 = wb5.add_sheet('Sheet5')
sheet6 = wb6.add_sheet('Sheet6')
sheet7 = wb7.add_sheet('Sheet7')
sheet8 = wb8.add_sheet('Sheet8')
sheet9 = wb9.add_sheet('Sheet9')
sheet10 = wb10.add_sheet('Sheet10')
sheet11 = wb11.add_sheet('Sheet10')
sheet12 = wb12.add_sheet('Sheet10')
sheet13 = wb13.add_sheet('Sheet10')

#Write the number of molecules you will evaluate (important for PCA)

#Some important counters for coding output into excel sheets 
index1=0
index2=0
index3=0
index4=0
index5=0
index6=0
index7=0
index8=0
index9=0
index10=0
index11=0
index12=0
index13=0

#BEGIN AUTOMATED CALCULATIONS, choose the range of QM9 to perform the calculations in
for idx in range(0,500):

# Load test set
    test_loader = spk.AtomsLoader(test, batch_size=100)
    converter = spk.data.AtomsConverter(device=device)
    
# Get properties of QM9
    at, props = qm9data.get_properties(idx)

    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()

    inputs = converter(at)
    print('Keys:', list(inputs.keys()))
    print('Truth:', props[QM9.U0].cpu().numpy()[0])

# Calculate prediction by best_model using schnetpack calculator 
    calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)
    at.set_calculator(calculator)

# Print energy prediction and index of molecule
    print('Prediction:', at.get_total_energy())
    print('idx',idx)

    
# Output atomization energies of each atom in the molecule
    import numpy as np

    from schnetpack.atomistic import output_modules

    atomenergies = (output_modules.yi)

    atomenergies = atomenergies.detach().numpy()
    atomenergies = atomenergies.astype(float)
    print(atomenergies[0][1])

    Z = props['_atomic_numbers']

    Z= Z.cpu().detach().numpy() 
     


# Write the xyz matrix for each molecule
    j = len(props['_positions'])
    print('number of atoms', j)
    for i in range(j):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 

# Extract atomenergies into a 1D array
    AE = np.zeros(len(z))
    for i in range(len(z)):
        AE[i] = atomenergies[0][i]
    print(AE)

# Take the average AE for each atom (for "1-atom only" PCA) and write to excel sheet

    numH = 0
    numC = 0
    numN = 0
    numO = 0
    numF = 0
    h=0
    c=0
    n=0
    o=0
    f=0
    for i in range(len(z)):
        atomicnumbers = props['_atomic_numbers'][i]
        if atomicnumbers == 1:
            numH = numH + atomenergies[0,i]
            h = h +1
        if atomicnumbers == 6:
            numC = numC + atomenergies[0,i]
            c = c +1
        if atomicnumbers == 7:
            numN = numN + atomenergies[0,i]
            n = n+1
        if atomicnumbers == 8:
            numO = numO + atomenergies[0,i]
            o = o + 1
        if atomicnumbers == 9:
            numF = numF + atomenergies[0,i]
            f = f + 1
    if numH != 0:
        aveH = numH[0]/h
    else:
        aveH = 0
    if numC != 0:
        aveC = numC[0]/c
    else:
        aveC = 0
    if numN != 0:
        aveN = numN[0]/n
    else:
        aveN = 0
    if numO != 0:
        aveO = numO[0]/o
    else:
        aveO = 0
    if numF !=0:
        aveF = numF[0]/f
    else:
        aveF = 0
    print(aveC)
    print(aveH)
    print(aveN)
    print(aveO)
    print(aveF)

# Write the target properties to an excel file for PCA analysis
    if numC != 0 and numO !=0 and numH!=0 and numN==0 and numF==0:
        sheet1.write(index1,0,idx)
        sheet1.write(index1,1,aveC)
        sheet1.write(index1,2,aveO)
        sheet1.write(index1,3,aveH)
        index1= index1+1
    wb1.save('./pca/atomwise-CHOonly.xls')
    read_file = pd.read_excel (r'./pca/atomwise-CHOonly.xls')
    read_file.to_csv (r'./pca/atomwise-CHOonly.csv', index = None, header=True)

    if numC!=0 and numH!=0 and numO==0 and numN==0 and numF==0:
        sheet3.write(index3,0,idx)
        sheet3.write(index3,1,aveC)
        sheet3.write(index3,2,aveH)
        index3= index3+1 
    wb3.save('./pca/atomwise-CHonly.xls')
    read_file = pd.read_excel (r'./pca/atomwise-CHonly.xls')
    read_file.to_csv (r'./pca/atomwise-CHonly.csv', index = None, header=True)
    
    if numC!=0 and numH!=0:
        sheet4.write(index4,0,idx)
        sheet4.write(index4,1,aveC)
        sheet4.write(index4,2,aveH)
        index4= index4+1 
    wb4.save('./pca/atomwise-CHall.xls')
    read_file = pd.read_excel (r'./pca/atomwise-CHall.xls')
    read_file.to_csv (r'./pca/atomwise-CHall.csv', index = None, header=True)
    
# Import internal stuff from schnet to do PCA on
    from schnetpack.representation.schnet import x
    embed=x
    from schnetpack.representation.schnet import v
    from schnetpack.nn.cfconv import W
    filt=W
    from schnetpack.nn.cfconv import y
    conv=y
    from schnetpack.nn.activations import soft

    
# convert embed to a 2D array!
    emb = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            emb[i,j] = embed[0][i][j]
#convert interaction layer to 2D array
    inter = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            inter[i,j] = v[0][i][j]
#convert soft to 2D array
    softact = np.zeros((len(z),15))
    for i in range(len(z)):
        for j in range(15):
            softact[i,j] = soft[0][i][j]

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            sheet5.write(index5,0,idx)
            for j in range(30):
                sheet5.write(index5,j+1,emb[place,j])
            index5=index5+1
            break
    wb5.save('./pca/embedding-O.xls')
    read_file = pd.read_excel (r'./pca/embedding-O.xls')
    read_file.to_csv (r'./pca/embedding-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            sheet6.write(index6,0,idx)
            for j in range(30):
                sheet6.write(index6,j+1,inter[place,j])
            index6=index6+1
            break
    wb6.save('./pca/interaction-O.xls')
    read_file = pd.read_excel (r'./pca/interaction-O.xls')
    read_file.to_csv (r'./pca/interaction-O.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            sheet7.write(index7,0,idx)
            for j in range(15):
                sheet7.write(index7,j+1,softact[place,j])
            index7=index7+1
            break
    wb7.save('./pca/softplus-O.xls')
    read_file = pd.read_excel (r'./pca/softplus-O.xls')
    read_file.to_csv (r'./pca/softplus-O.csv', index = None, header=True)
    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            sheet8.write(index8,0,idx)
            for j in range(30):
                sheet8.write(index8,j+1,emb[place,j])
            index8=index8+1
            break
    wb8.save('./pca/embedding-H.xls')
    read_file = pd.read_excel (r'./pca/embedding-H.xls')
    read_file.to_csv (r'./pca/embedding-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            sheet9.write(index9,0,idx)
            for j in range(30):
                sheet9.write(index9,j+1,inter[place,j])
            index9=index9+1
            break
    wb9.save('./pca/interaction-H.xls')
    read_file = pd.read_excel (r'./pca/interaction-H.xls')
    read_file.to_csv (r'./pca/interaction-H.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            sheet10.write(index10,0,idx)
            for j in range(15):
                sheet10.write(index10,j+1,softact[place,j])
            index10=index10+1
            break
    wb10.save('./pca/softplus-H.xls')
    read_file = pd.read_excel (r'./pca/softplus-H.xls')
    read_file.to_csv (r'./pca/softplus-H.csv', index = None, header=True)

    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            sheet11.write(index11,0,idx)
            for j in range(30):
                sheet11.write(index11,j+1,emb[place,j])
            index11=index11+1
            break
    wb11.save('./pca/embedding-N.xls')
    read_file = pd.read_excel (r'./pca/embedding-N.xls')
    read_file.to_csv (r'./pca/embedding-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            sheet12.write(index12,0,idx)
            for j in range(30):
                sheet12.write(index12,j+1,inter[place,j])
            index12=index12+1
            break
    wb12.save('./pca/interaction-N.xls')
    read_file = pd.read_excel (r'./pca/interaction-N.xls')
    read_file.to_csv (r'./pca/interaction-N.csv', index = None, header=True)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 7:
            place=i
            sheet13.write(index13,0,idx)
            for j in range(15):
                sheet13.write(index13,j+1,softact[place,j])
            index13=index13+1
            break
    wb13.save('./pca/softplus-N.xls')
    read_file = pd.read_excel (r'./pca/softplus-N.xls')
    read_file.to_csv (r'./pca/softplus-N.csv', index = None, header=True)
    



Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -1101.4878
hello
Prediction: -1101.466796875
idx 0
[-17.07192421]
number of atoms 5
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613
[-1033.17919922   -17.07192421   -17.07192421   -17.07192421
   -17.07192421]
-1033.17919921875
-17.071924209594727
0
0
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -1538.1477
hello
Prediction: -1538.0333251953125
idx 1
[-16.49673271]
number of atoms 4
N -0.03160822 0.021159047 0.060332563
H 0.026075294 -0.9904034 -0.029608395
H 0.9246072 0.35579658 -0.030988995
H -0.5114599 0.34058344 -0.77777386
[-1488.54309082   -16.49673271   -16.4967308    -16.4968338 ]
0
-16.49676577250163
-1488.5430908203125
0
0
Keys: ['_atomic_numbers'

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -5254.201
hello
Prediction: -5254.24755859375
idx 17
[-1034.05407715]
number of atoms 10
C -0.048322067 1.4560183 -0.17357557
C -0.09049219 -0.05335558 0.00041801704
C 1.2518378 -0.74642915 0.16791351
O -1.1316519 -0.66721845 0.0053270245
H 0.5488151 1.7222008 -1.0532273
H -1.0606242 1.8458552 -0.28479958
H 0.43577582 1.926221 0.69001687
H 1.7799013 -0.3491923 1.0423096
H 1.1037049 -1.8204619 0.284567
H 1.8916062 -0.5539819 -0.70093095
[-1033.69018555 -1034.05407715 -1033.69018555 -2048.23413086
   -17.4490509    -17.39125633   -17.44905663   -17.44905281
   -17.39125633   -17.44905853]
-1033.8114827473958
-17.42978858947754
0
-2048.234130859375
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -5691.5146
hello
Prediction: -5691.69140625
idx 18
[-1033.68103027]
number of atoms 9
C -0.041906178 1.45598

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -5219.441
hello
Prediction: -5219.70751953125
idx 32
[-1033.59069824]
number of atoms 8
O 0.8357715 1.3373044 -0.00080194796
C 0.8575883 -0.084277265 -0.008656541
C -0.5077634 -0.5948881 0.0016466676
C -1.623125 -1.0397791 0.009920608
H 1.7503725 1.6372886 -0.007630262
H 1.3756938 -0.4718312 -0.90070695
H 1.392608 -0.48135367 0.86910504
H -2.6118317 -1.4274471 0.017277308
[-2046.33337402 -1033.59069824 -1034.9239502  -1034.77416992
   -17.18771553   -17.55850029   -17.55845642   -17.78078461]
-1034.4296061197917
-17.521364212036133
0
-2046.3333740234375
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -5657.704
hello
Prediction: -5657.91162109375
idx 33
[-1033.51000977]
number of atoms 7
O 0.8167186 1.3475205 -0.00062532997
C 0.86702985 -0.06354135 -0.008641366
C -0.50337374 -0.5867032 0.0016643667
N

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -5252.875
hello
Prediction: -5253.19873046875
idx 43
[-1033.58935547]
number of atoms 10
C -0.5051477 1.4701715 0.23591569
C -0.5161165 -0.03155866 0.11930383
C 0.2842813 -0.74945444 -0.8804299
O 0.6834387 -0.71516526 0.49098954
H 0.42997557 1.8810065 -0.15465757
H -1.3402644 1.9064294 -0.32289335
H -0.6035613 1.777829 1.282179
H -1.4245102 -0.5216416 0.47502917
H -0.044245012 -1.7201506 -1.249905
H 0.91669863 -0.18413898 -1.5645137
[-1033.6583252  -1033.58935547 -1033.63720703 -2047.60046387
   -17.36555481   -17.37056732   -17.37960243   -17.63806534
   -17.48276711   -17.47636986]
-1033.6282958984375
-17.452154477437336
0
-2047.6004638671875
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -4711.5913
hello
Prediction: -4711.80419921875
idx 44
[-1490.07116699]
number of atoms 11
C -0.5746446 1.3396

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -6355.5083
hello
Prediction: -6355.78466796875
idx 54
[-1033.51660156]
number of atoms 15
C -0.023475222 1.5470222 0.007835588
C -0.009847433 0.011347052 0.02487002
C 0.7464621 -0.5140713 1.2539729
C -1.435043 -0.54501116 -0.014851667
O 0.6109895 -0.47931674 -1.1702621
H 0.99869883 1.9445285 0.0091197025
H -0.5295119 1.9113454 -0.8907494
H -0.53706336 1.9531459 0.88549185
H 0.2663909 -0.19733478 2.1857443
H 0.78473985 -1.6069968 1.2365339
H 1.7760539 -0.13640866 1.2671385
H -1.9534636 -0.19478653 -0.9119744
H -1.4142915 -1.6382699 -0.039329913
H -2.0034225 -0.22627586 0.8635737
H 1.5161966 -0.15055977 -1.1857938
[-1033.78625488 -1033.51660156 -1033.78637695 -1033.7947998
 -2046.74816895   -17.47485352   -17.3965683    -17.40463257
   -17.40463257   -17.39656639   -17.47486496   -17.42051697
   -17.42050743   -17.43721581   -17.32298279]
-1033.7210083007812
-17.415334129

...

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -7359.0786
hello
Prediction: -7359.33837890625
idx 455
[-1033.87597656]
number of atoms 14
O 0.23058486 1.8788811 0.40447217
C 0.37167057 0.48769274 0.7023757
C -0.9724473 -0.29103136 0.76123124
C -1.0386082 -1.1480868 -0.5294427
C 0.37325975 -1.0841446 -1.0607744
C 1.1424227 -0.23822105 -0.37590596
H -0.04802964 1.9447361 -0.5158251
H 0.88059783 0.4787132 1.6716287
H -0.9812285 -0.9422367 1.6402103
H -1.8069491 0.40645152 0.8652843
H -1.3538723 -2.1774445 -0.32348093
H -1.7540452 -0.75918823 -1.2666951
H 0.695914 -1.6674936 -1.9172467
H 2.181764 -0.011430418 -0.5858498
[-2046.58947754 -1033.87597656 -1033.94946289 -1033.97595215
 -1034.57092285 -1034.74475098   -17.38555908   -17.7118187
   -17.63005066   -17.62726784   -17.64001465   -17.62120247
   -17.9641037    -18.05151749]
-1034.2234130859374
-17.703941822052002
0
-2046.5894775390625
0
Keys: ['_atomic_numbers', '

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -6413.5664
hello
Prediction: -6413.744140625
idx 463
[-1034.24987793]
number of atoms 18
C -1.1843599 1.4251418 -0.466773
C -0.98605543 -0.07603658 -0.42967626
C 0.35017824 -0.7420724 -0.6995966
C 1.6085883 0.033031665 -1.03084
C -0.25242615 -0.7873836 0.69220126
C 0.34865773 -0.061333716 1.8778282
H -0.32745123 1.9783897 -0.07571862
H -1.353577 1.7710451 -1.4928913
H -2.0581405 1.717572 0.12668003
H -1.8456745 -0.6169605 -0.819522
H 0.2836163 -1.6782992 -1.2496365
H 1.6857606 0.9750891 -0.4832299
H 2.5024478 -0.5550089 -0.79333085
H 1.6483129 0.27403712 -2.0992677
H -0.6765938 -1.7504177 0.96817636
H 0.77768564 0.9074739 1.6120776
H -0.40878505 0.11937762 2.6490636
H 1.1476701 -0.6560595 2.3350825
[-1033.69909668 -1034.24987793 -1034.25036621 -1033.69909668
 -1034.24963379 -1033.69909668   -17.38096619   -17.38600922
   -17.38614273   -17.81218529   -17.81234741   -17.

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -7357.894
hello
Prediction: -7358.15234375
idx 472
[-1034.13842773]
number of atoms 14
O 0.14625268 1.8439193 -0.14986776
C 0.19197103 0.4603829 -0.14008063
C -0.08674813 -0.3654912 -1.363675
C -0.9844084 -0.50460494 -0.15071884
C -0.2902541 -1.3818158 0.89812857
C 0.9042409 -0.3683567 0.9362908
H 0.861958 2.1671352 -0.70802873
H -0.52640057 0.1952983 -2.18261
H 0.5691388 -1.1801358 -1.6667178
H -2.0454872 -0.29653984 -0.20922631
H -0.82504094 -1.4331702 1.8487399
H -0.049353607 -2.4000278 0.57556546
H 1.9012082 -0.7496903 0.6895987
H 0.9526717 0.16689695 1.8872194
[-2046.79406738 -1034.13842773 -1034.10412598 -1034.28564453
 -1033.96923828 -1034.00268555   -17.36339188   -17.68012619
   -17.64569473   -17.78516006   -17.58215904   -17.56378365
   -17.61554146   -17.62272644]
-1034.1000244140625
-17.607322931289673
0
-2046.7940673828125
0
Keys: ['_atomic_numbers', '_pos

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -8200.034
hello
Prediction: -8200.13671875
idx 484
[-1033.74462891]
number of atoms 10
N 1.1426964 2.249567 -0.17271355
C 0.8936848 1.0587627 0.14150853
O 0.4802528 0.19244131 -0.8236413
C 0.21478726 -1.1324109 -0.401962
C -1.0366724 -1.2501303 0.36412385
N -2.0114503 -1.3419987 0.9765697
H 1.4471775 2.7759755 0.6433442
H 0.9724776 0.60239065 1.1368537
H 0.1362287 -1.7310208 -1.3125079
H 1.038049 -1.539481 0.19961008
[-1489.57788086 -1033.74462891 -2047.71020508 -1033.5958252
 -1034.50292969 -1491.14123535   -17.36694145   -17.51664925
   -17.47432137   -17.50670433]
-1033.9477945963542
-17.466154098510742
-1490.3595581054688
-2047.710205078125
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -7325.4937
hello
Prediction: -7325.65625
idx 485
[-1034.01953125]
number of atoms 12
O -1.8138177 -1.7621415 

Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -6314.7803
hello
Prediction: -6314.7734375
idx 495
[-1035.17236328]
number of atoms 12
C -0.0454974 3.3421922 0.018617
C -0.025434269 1.8903415 0.010177316
C -0.00896302 0.6801152 0.003359858
C 0.009312059 -0.68010736 -0.0038843928
C 0.02556782 -1.8903389 -0.010294797
C 0.04513767 -3.3422 -0.018135287
H 0.9703522 3.7536104 0.011179011
H -0.5701666 3.737393 -0.85877174
H -0.5531496 3.7272625 0.9103954
H -0.9708476 -3.7532842 -0.010665851
H 0.55277544 -3.7277853 -0.9096983
H 0.5695721 -3.7372286 0.8594715
[-1033.70739746 -1035.17236328 -1036.22497559 -1036.22497559
 -1035.17236328 -1033.70739746   -17.42728615   -17.42728615
   -17.42729568   -17.42728615   -17.42729568   -17.42728424]
-1035.034912109375
-17.42728900909424
0
0
0
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask']
Truth: -6314.0312
hello
Prediction